In [36]:
import os
import pandas as pd

In [30]:
os.chdir("/Users/Dippies/CODE PACKT - EML/Chapter 3/Logistic Regression")
os.chdir("/Users/Dippies/CODE PACKT - EML/Chapter 7")
os.getcwd()

'/Users/Dippies/CODE PACKT - EML/Chapter 7'

In [33]:
df_creditdata = pd.read_csv("UCI_Credit_Card.csv")

In [34]:
#dropping the ID column, as it would not be  required
df_creditdata.drop(["ID"],axis=1,inplace=True)

In [37]:
from sklearn.model_selection import train_test_split

#create train and test subset
X = df_creditdata.iloc[:,0:23]
Y = df_creditdata['default.payment.next.month']

# Create train & test sets
X_train, X_test, Y_train, Y_test = \
train_test_split(X, Y, test_size=0.30, random_state=1)

In [69]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(21000, 23)
(21000,)
(9000, 23)
(9000,)


#### We will build 3 base learners using Logistic Regression, Decision Tree and KNN
#### We will also import GaussianNB to use the same for building our meta-learner

In [12]:
# for the base learners
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# For the meta-learner
from sklearn.naive_bayes import GaussianNB

In [26]:
# The base learners
base_learner_1 = LogisticRegression()
base_learner_2 = KNeighborsClassifier(n_neighbors=1)
base_learner_3 = DecisionTreeClassifier()

# The meta-learner
meta_learner = GaussianNB()

In [149]:
# Now we train a list of models
result_1 = base_learner_1.fit(X_train, Y_train)
result_2 = base_learner_2.fit(X_train, Y_train)
result_3 = base_learner_3.fit(X_train, Y_train)

In [150]:
# We then use the models to make predictions and create a new stacked dataset
predicted_1 = result_1.predict(X_train)
predicted_2 = result_2.predict(X_train)
predicted_3 = result_3.predict(X_train)

In [152]:
import numpy as np
prediction_stack = np.dstack([predicted_1, predicted_2, predicted_3])

# Now we stack the actual outcomes i.e. Y_Test with the prediction_stack
final_stack = np.dstack([prediction_stack, Y_train])

In [154]:
final_stack

array([[[0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 1, 1, 1],
        ..., 
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [0, 0, 0, 0]]])

In [156]:
stacked_dataframe = pd.DataFrame(final_stack[0,0:21000], columns='LR KNN DT Y_TRAIN'.split())
stacked_dataframe.head(5)

,LR,KNN,DT,Y_TRAIN
0,0,0,0,0
1,0,0,0,0
2,0,1,1,1
3,0,1,1,1
4,0,0,0,0


In [ ]:
stacked_model = meta_learner.fit()

In [145]:
# BASE LEARNER 1
result_1 = base_learner_1.fit(X_train, Y_train)
predicted_1 = result_1.predict(X_test)
base_learner_1.score(X_test,Y_test)

0.77919047619047621

In [43]:
# BASE LEARNER 2
result_2 = base_learner_2.fit(X_train, Y_train)
predicted_2 = result_2.predict(X_test)
base_learner_2.score(X_test,Y_test)

0.69344444444444442

In [44]:
# BASE LEARNER 2
result_3 = base_learner_3.fit(X_train, Y_train)
predicted_3 = result_3.predict(X_test)
base_learner_3.score(X_test,Y_test)

0.72677777777777774

In [148]:
from sklearn import model_selection
for learners, label in zip([base_learner_1, base_learner_2, base_learner_3], 
                      ['Logistic Regression', 
                       'KNN', 
                       'Decision Tree']):
    learners.fit(X_train, Y_train)
    scores = learners.score(X_train, Y_train)
    
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

Accuracy: 0.78 (+/- 0.00) [Logistic Regression]
Accuracy: 1.00 (+/- 0.00) [KNN]
Accuracy: 1.00 (+/- 0.00) [Decision Tree]


In [46]:
# Predicted results
prediction_from_base_learner_1 = base_learner_1.predict(X_test)
prediction_from_base_learner_2 = base_learner_2.predict(X_test)
prediction_from_base_learner_3 = base_learner_3.predict(X_test)

In [48]:
type(prediction_from_base_learner_1)

numpy.ndarray

In [56]:
import numpy as np
prediction_stack = np.dstack([prediction_from_base_learner_1, prediction_from_base_learner_2, prediction_from_base_learner_3])

In [117]:
type(prediction_stack)

numpy.ndarray

In [52]:
# Now we stack the actual outcomes i.e. Y_Test with the prediction_stack
final_stack = np.dstack([prediction_stack, Y_test])

In [53]:
final_stack

array([[[0, 0, 0, 0],
        [0, 1, 0, 1],
        [0, 0, 0, 0],
        ..., 
        [0, 0, 0, 0],
        [0, 0, 1, 0],
        [0, 1, 0, 0]]])

In [143]:
stacked_dataframe = pd.DataFrame(final_stack[0,0:9000], columns='LR KNN DT Y_TEST'.split())
stacked_dataframe.head(5)

,LR,KNN,DT,Y_TEST
0,0,0,0,0
1,0,1,0,1
2,0,0,0,0
3,0,1,1,1
4,0,0,0,0
